# XGBoost
# Initial point: Artificial Neural Network file

In [14]:
#importing libraries

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [15]:
#importing dataset
dataset=pd.read_csv('./../0. DataSets/Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0,1,1,1,"101,349",1
1,2,15647311,Hill,608,Spain,Female,41,1,"83,808",1,0,1,"112,543",0
2,3,15619304,Onio,502,France,Female,42,8,"159,661",3,1,0,"113,932",1
3,4,15701354,Boni,699,France,Female,39,1,0,2,0,0,"93,827",0
4,5,15737888,Mitchell,850,Spain,Female,43,2,"125,511",1,1,1,"79,084",0


In [16]:
#Faz com que os displays (tanto do pandas como dos arrays tenham notacao decimal)
dataset.round()
pd.options.display.float_format = '{:,.0f}'.format
float_formatter = lambda x: "%.2f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})

## Business Problem explanation:

In [17]:
#we have a list of customers from a bank with loads of different independent variables.
#based on these variables we want to predict the last column, that is, if the customer left/exited the bank or not

In [18]:
#Create X matrix with all the independent variables
#Create y vector with all the dependent variables
X=dataset.iloc[:,3:13].values #ignored the first columns, ie. customer id, surname
y=dataset.iloc[:,13].values

In [19]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [20]:
#Encoding categorical data
#We only have 2 categorical variables: Geography and Gender. We have to create 2 different encoders

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

#encode Geography, that means turn a word and turn into a number. On this case France will be turned into 0 and Spain will be turned into 2
labelencoder_X_1=LabelEncoder()
X[:,1]=labelencoder_X_1.fit_transform(X[:,1])

#encode Gender
labelencoder_X_2=LabelEncoder()
X[:,2]=labelencoder_X_2.fit_transform(X[:,2])

#Vamos onehotencode coluna 1. Que e geography.
#One hot encoded columns passam para o inicio da matrix (deixa de ser o credit score que agora esta na posicao 3)
onehotencoder=OneHotEncoder(categorical_features=[1])
X=onehotencoder.fit_transform(X).toarray()


In [21]:
#como fizemos one hot encode e ficamos com tres colunas (correnspondendo a spain,france e outro pais)
#podemos eliminar uma coluna para nao cair na dummy variable trap. ficando so com 2 dummy variables
X=X[:,1:]
X

array([[0.00, 0.00, 619.00, ..., 1.00, 1.00, 101348.88],
       [0.00, 1.00, 608.00, ..., 0.00, 1.00, 112542.58],
       [0.00, 0.00, 502.00, ..., 1.00, 0.00, 113931.57],
       ...,
       [0.00, 0.00, 709.00, ..., 0.00, 1.00, 42085.58],
       [1.00, 0.00, 772.00, ..., 1.00, 0.00, 92888.52],
       [0.00, 0.00, 792.00, ..., 1.00, 0.00, 38190.78]])

In [22]:
#Now, we have to split the dataset into the training set/test set
# from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)


# A partir daqui e a parte nova do XGBoost

In [23]:
#Fitting XGBoost to the Training set
from xgboost import XGBClassifier


#dentro deste XGClassifier existem parametros que podem ser ajustados
#posso sempre usar o gridsearch que aprendi na aula anterior para os fine tune..
classifier=XGBClassifier()

classifier.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [25]:
#fazer a confusion matrix

#agora que treinei o modelo vou usar a matrix X_test para o testar.
y_pred=classifier.predict(X_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)

cm

/home/xbuns/anaconda3/envs/my_env_1/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([[1521,   74],
       [ 197,  208]])

In [27]:
#Applying k-Fold cross validation
from sklearn.model_selection import cross_val_score
accuracies=cross_val_score(estimator=classifier, X=X_train,y=y_train,cv=10)

#parti o X_train em 10 subsets iguais e testei o modelo. o array accuracies da a accuracy do 
#modelo em cada um destes 10 subsets

accuracies.mean()  #tirar a media do vector
# accuracies.std()

#isto da a accuracy do modelo (foi 86%) e da o desvio (1%)

/home/xbuns/anaconda3/envs/my_env_1/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/xbuns/anaconda3/envs/my_env_1/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/xbuns/anaconda3/envs/my_env_1/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/xbuns/anaconda3/envs/my_env_1/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: Deprec

0.8629994451163204